In [69]:
import os

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [222]:
# creamos las carpetas si no estan
path_dirs = ['urbano/', 'interurbano/', 'media-distancia/']
for i in path_dirs:
    if not os.path.isdir(i):
        os.mkdir(i)  
        
yml = '''---
layout: default
permalink: /{}{}  
---

'''        

# url donde esta la data
url = 'http://satcrc.com.ar/horarios-y-recorridos/'
# hacemos le requests y parseamos el html
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

tables_titles = soup.find_all('h2', attrs={'class':'tablepress-table-name'})

# por otro lado, levantamos todas las tablas con pandas
dfs = pd.read_html(url)
# completamos los nan con un str vacio
for df in dfs:
    df.fillna('', inplace=True)
    col_names = df.columns
    df.columns = [i if not 'Unnamed' in i else '' for i in df.columns]
    
# Eliminamos la ultima tabla vacia del 8 
(tables_titles.pop(27), dfs.pop(27))

(<h2 class="tablepress-table-name tablepress-table-name-id-64">Linea 08 - Horario Especial</h2>,
   Plaza H. Nuevo
 0               
 1               
 2               
 3               )

In [223]:
# Spliteamos los titulos de las tablas para saber
# cuales hacen referencia a la misma

titulos = []
for i in range(len(dfs)):
    titulo_raw = tables_titles[i].text
    
    if 'Las Higueras' in titulo_raw:
        titulo = 'las-higueras'
        titulos.append(titulo)
        continue
    
    if 'Holmberg' in titulo_raw:
        titulo = 'holmberg'
        titulos.append(titulo)
        continue
        
    if 'Villa María' in titulo_raw:
        titulo = 'villa-maria'
        titulos.append(titulo)
        continue
        
    if 'Cnel. Moldes' in titulo_raw:
        titulo = 'cnel-moldes'
        titulos.append(titulo)
        continue
        
    titulo = titulo_raw.lower()
    titulo = titulo.split('-')[0]
    if titulo[-1] == ' ':
        titulo = titulo[:-1]
    
    for l in ['línea ', 'linea ']:
        if l in titulo:
            titulo = titulo.split(l)[1]    

    if 'de ' in titulo:
        titulo = titulo.split('de')[0]
        titulo = ''.join(titulo.split(' '))

        
        
    titulo = titulo.replace(' ', '-')
        
    if len(titulo) == 1 and titulo != 'a':
        titulo = '0' + titulo
    
    titulos.append(titulo)

In [224]:
for i in range(61):
    if i < 47:
        dir_out = path_dirs[0]
    elif i < 53:
        dir_out = path_dirs[1]
    elif i < 61:
        dir_out = path_dirs[2]
    
    path_file = dir_out + titulos[i] + '.html'
    
    if not os.path.isfile(path_file):
        new_file = True
    else:
        new_file = False
    
    with open(path_file, 'a') as file:
        if new_file:
            file.write(yml.format(dir_out, titulos[i]))
        file.write('<h2> ' + tables_titles[i].text + ' </h2>\n')    
        file.write(dfs[i].to_html(index=False, border=0))
        file.write('\n')